In [15]:
using System.Diagnostics;
using System.Diagnostics.Tracing;

In [16]:
[EventSource(Name = "My-CustomMetricsEventSource-Minimal")]
public sealed class CustomMetricsEventSource : EventSource
{
	private EventCounter methodDurationCounter;
	private IncrementingEventCounter methodCallCounter;
	private IncrementingPollingCounter incrPollingCounter;
	private PollingCounter pollingCounter;

	private Dictionary<string, EventCounter> dynamicCounters =
		new Dictionary<string, EventCounter>();

	public static CustomMetricsEventSource Log = new CustomMetricsEventSource();

	public CustomMetricsEventSource()
	{
		methodDurationCounter = new EventCounter(nameof(methodDurationCounter), this);
		methodDurationCounter.AddMetadata("Environment", "Production");
		methodDurationCounter.AddMetadata("Customer", "Ibis");
		methodCallCounter = new IncrementingEventCounter(nameof(methodCallCounter), this);
		incrPollingCounter = new IncrementingPollingCounter(nameof(incrPollingCounter), this, () => new Random().NextDouble());
		incrPollingCounter.AddMetadata("Environment", "Staging");
		pollingCounter = new PollingCounter(nameof(pollingCounter), this, ()=> new Random().NextDouble());
	}

	public void ReportMethodDurationInMs(long milliseconds)
	{
		methodDurationCounter.WriteMetric(milliseconds);
		methodCallCounter.Increment();
	}

	public void ReportMetric(string name, float value)
	{
		if (!dynamicCounters.TryGetValue(name, out EventCounter counterInstance))
		{
			counterInstance = new EventCounter(name, this);
			dynamicCounters.Add(name, counterInstance);
		}
		counterInstance.WriteMetric(value);
	}
}

In [18]:
internal class CustomMetricsEventListener : EventListener
{
	protected override void OnEventWritten(EventWrittenEventArgs eventData)
	{
        var isEventCounter = eventData.EventName == "EventCounters";
		if(!isEventCounter)
			return;
		
		var payload = (IDictionary<string, object>)eventData.Payload[0];
		Console.WriteLine(string.Join(", ", payload.Select(p => $"{p.Key}: {p.Value}")));
	}
}

Unhandled exception: (5,17): error CS1061: 'EventWrittenEventArgs' does not contain a definition for 'IsEventCounter' and no accessible extension method 'IsEventCounter' accepting a first argument of type 'EventWrittenEventArgs' could be found (are you missing a using directive or an assembly reference?)

In [19]:
using System.Diagnostics;
using System.Threading;

static void SleepingBeauty(int sleepTimeInMs)
{
	var stopwatch = Stopwatch.StartNew();

	Thread.Sleep(sleepTimeInMs);

	stopwatch.Stop();

	CustomMetricsEventSource.Log.ReportMethodDurationInMs(stopwatch.ElapsedMilliseconds);
	CustomMetricsEventSource.Log.ReportMetric("someCounter", DateTime.Now.Millisecond);
}

In [20]:
var reader = new CustomMetricsEventListener();
	var arguments = new Dictionary<string, string>
		{
			{"EventCounterIntervalSec", "1"}
		};
	reader.EnableEvents(CustomMetricsEventSource.Log, EventLevel.LogAlways, EventKeywords.All, arguments);

	var random = new Random();
	for (int i = 0; i <= 10000000; i++)
	{
		SleepingBeauty(random.Next(10, 200));
	}

	Console.ReadLine();

Unhandled exception: (1,18): error CS0246: The type or namespace name 'CustomMetricsEventListener' could not be found (are you missing a using directive or an assembly reference?)